# QNNCV: Quantum Neural Networks for Continuous Variables - Google Colab Template

**Continuous Variable Quantum Generative Adversarial Networks**

This notebook provides a complete template for running QNNCV experiments in Google Colab with:
- Automatic environment setup and package installation
- GPU acceleration (when available) (Need to be activated in Runtime)
- Repository cloning and module imports
- Ready-to-use training templates
- Comprehensive visualization tools

---

**Quick Start:**
1. Run all cells in order
2. Modify the training parameters in Section 6
3. Execute your quantum GAN experiments

**Hardware Recommendation:** Use GPU runtime for faster training

---


**Environment Detection**

In [ ]:
# Environment detection and basic setup
import sys
import os
import platform
import subprocess

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

print("=" * 60)
print("QNNCV ENVIRONMENT SETUP")
print("=" * 60)
print(f"Running in Google Colab: {IN_COLAB}")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")

if IN_COLAB:
    print("Colab environment detected - optimizing for cloud execution")
    
    # Check GPU availability
    gpu_info = !nvidia-smi
    if gpu_info:
        print("GPU detected - will enable acceleration")
    else:
        print("No GPU detected - using CPU (consider switching to GPU runtime)")
else:
    print("Local environment detected")

print("=" * 60)


**Package Installation**

In [ ]:
# Package installation optimized for Colab
print("Installing required packages...")

# Core quantum computing packages
!pip install -q strawberryfields
!pip install -q tensorflow>=2.8.0

# Scientific computing packages
!pip install -q numpy scipy matplotlib seaborn
!pip install -q pandas scikit-learn

# Visualization and progress tracking
!pip install -q plotly tqdm ipywidgets

# Additional utilities
!pip install -q psutil  # For memory monitoring

print("Package installation completed!")

# Verify key installations
try:
    import strawberryfields as sf
    import tensorflow as tf
    import numpy as np
    import matplotlib.pyplot as plt
    print(f"✓ Strawberry Fields version: {sf.__version__}")
    print(f"✓ TensorFlow version: {tf.__version__}")
    print(f"✓ NumPy version: {np.__version__}")
    print("All core packages installed successfully!")
except ImportError as e:
    print(f"Installation error: {e} :(")
    print("Please restart runtime and try again")


**GPU Configuration**

In [ ]:
import tensorflow as tf
import os

print("Configuring TensorFlow and GPU settings...")

# Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ GPU(s) detected: {len(gpus)}")
    for i, gpu in enumerate(gpus):
        print(f"  GPU {i}: {gpu}")
    
    # Configure GPU memory growth to avoid allocation issues
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✓ GPU memory growth configured")
    except RuntimeError as e:
        print(f"GPU configuration warning: {e}")
else:
    print("⚠ No GPU detected - using CPU")
    print("For faster training, consider switching to GPU runtime:")
    print("Runtime → Change runtime type → Hardware accelerator → GPU")

# Configure TensorFlow for quantum computing
tf.config.run_functions_eagerly(False)  # Disable eager execution for performance

# Set random seeds for reproducibility
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)

print(f"✓ TensorFlow configured for quantum computing")
print(f"✓ Random seeds set for reproducibility")

# Memory monitoring setup
import psutil
print(f"✓ Available RAM: {psutil.virtual_memory().available / 1e9:.1f} GB")


**Repository Setup**

In [ ]:
# Repository setup
import os
import sys

# Define repository URL and name
REPO_URL = "https://github.com/MarcoAntonioCartia/QNNCV"
REPO_NAME = "QNNCV"

print("Setting up QNNCV repository...")

# Check if repository already exists
if os.path.exists(REPO_NAME):
    print(f"Repository {REPO_NAME} already exists - updating...")
    %cd {REPO_NAME}
    !git pull
else:
    print(f"Cloning repository from {REPO_URL}...")
    !git clone {REPO_URL}
    %cd {REPO_NAME}

# Verify repository structure
print("\nRepository structure:")
!ls -la

# Add src directory to Python path
repo_path = os.getcwd()
src_path = os.path.join(repo_path, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✓ Added {src_path} to Python path")

print(f"✓ Working directory: {os.getcwd()}")
print(f"✓ Repository setup complete")


**Import Modules**

In [ ]:
# Core scientific computing imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import strawberryfields as sf
from scipy.stats import wasserstein_distance
import time
import warnings
from tqdm.notebook import tqdm

# Configure matplotlib for Colab
%matplotlib inline
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Configure seaborn
sns.set_palette("husl")

print("Importing QNNCV modules...")

try:
    # Import QNNCV quantum components
    from models.generators.quantum_sf_generator import QuantumSFGenerator
    from models.discriminators.quantum_sf_discriminator import QuantumSFDiscriminator
    from training.qgan_sf_trainer import QGANSFTrainer
    
    # Import utilities
    from utils.warning_suppression import enable_clean_training
    
    print("✓ All QNNCV modules imported successfully")
    
    # Enable clean training environment
    enable_clean_training()
    print("✓ Clean training environment enabled")
    
except ImportError as e:
    print(f"Import error: {e} :(")
    print("\nTroubleshooting:")
    print("1. Verify repository was cloned correctly")
    print("2. Check that src/ directory exists")
    print("3. Ensure all required files are present")
    
    # Show current directory contents for debugging
    print("\nCurrent directory contents:")
    !ls -la
    
    if os.path.exists('src'):
        print("\nSrc directory contents:")
        !ls -la src/

print("\n" + "="*50)
print("ENVIRONMENT SETUP COMPLETE :)")
print("="*50)
print("Ready for quantum GAN experiments!")


**Functionality Test**

In [ ]:
print("Running functionality tests...")

# Test 1: Basic TensorFlow and Strawberry Fields
print("\n1. Testing TensorFlow and Strawberry Fields integration...")
try:
    # Create simple SF program
    prog = sf.Program(1)
    eng = sf.Engine("tf", backend_options={"cutoff_dim": 5})
    
    with prog.context as q:
        sf.ops.Dgate(0.5) | q[0]
        sf.ops.MeasureHomodyne(0) | q[0]
    
    result = eng.run(prog)
    print(f"✓ SF-TensorFlow integration working")
    print(f"  Sample measurement: {result.samples[0]:.3f}")
    
except Exception as e:
    print(f"***SF-TensorFlow test failed: {e}***")

# Test 2: Quantum components
print("\n2. Testing quantum components...")
try:
    # Test generator
    generator = QuantumSFGenerator(n_modes=2, latent_dim=2, layers=1, cutoff_dim=5)
    z_test = tf.random.normal([2, 2])
    samples = generator.generate(z_test)
    print(f"✓ Generator working - output shape: {samples.shape}")
    
    # Test discriminator
    discriminator = QuantumSFDiscriminator(n_modes=1, input_dim=2, layers=1, cutoff_dim=5)
    x_test = tf.random.normal([2, 2])
    probs = discriminator.discriminate(x_test)
    print(f"✓ Discriminator working - output shape: {probs.shape}")
    
except Exception as e:
    print(f"***Quantum components test failed: {e}***")

# Test 3: Gradient computation
print("\n3. Testing gradient computation...")
try:
    with tf.GradientTape() as tape:
        z = tf.random.normal([1, 2])
        output = generator.generate(z)
        loss = tf.reduce_mean(tf.square(output))
    
    gradients = tape.gradient(loss, generator.trainable_variables)
    non_none_grads = [g for g in gradients if g is not None]
    
    print(f"✓ Gradients computed: {len(non_none_grads)}/{len(gradients)} non-None")
    print(f"  Loss value: {loss:.4f}")
    
except Exception as e:
    print(f"***Gradient test failed: {e}***")

print("\n" + "="*40)
print("FUNCTIONALITY TESTS COMPLETE")
print("="*40)
print("System ready for quantum GAN training!")


**Data Preparation**

In [ ]:
def create_sample_data(n_samples=1000, data_type="gaussian_mixture"):
    """
    Create sample data for quantum GAN training.
    
    Args:
        n_samples: Number of samples to generate
        data_type: Type of data ('gaussian_mixture', 'spiral', 'moons')
    
    Returns:
        tf.Tensor: Training data [n_samples, 2]
    """
    np.random.seed(42)
    
    if data_type == "gaussian_mixture":
        # Two Gaussian clusters
        cluster1 = np.random.normal([1.5, 1.5], 0.4, (n_samples//2, 2))
        cluster2 = np.random.normal([-1.5, -1.5], 0.4, (n_samples//2, 2))
        data = np.vstack([cluster1, cluster2])
        
    elif data_type == "spiral":
        # Spiral pattern
        t = np.linspace(0, 4*np.pi, n_samples)
        r = t / (4*np.pi)
        x = r * np.cos(t) + np.random.normal(0, 0.1, n_samples)
        y = r * np.sin(t) + np.random.normal(0, 0.1, n_samples)
        data = np.column_stack([x, y])
        
    elif data_type == "moons":
        # Two moons pattern
        from sklearn.datasets import make_moons
        data, _ = make_moons(n_samples=n_samples, noise=0.1, random_state=42)
        
    else:
        raise ValueError(f"Unknown data_type: {data_type}")
    
    # Normalize to [-1, 1] range for quantum stability
    data = data / np.max(np.abs(data))
    
    return tf.constant(data, dtype=tf.float32)

# Create sample data
print("Creating sample training data...")

# Choose data type: 'gaussian_mixture', 'spiral', 'moons'
DATA_TYPE = "gaussian_mixture"  # MODIFY THIS
N_SAMPLES = 1000  # MODIFY THIS

training_data = create_sample_data(n_samples=N_SAMPLES, data_type=DATA_TYPE)

print(f"✓ Training data created: {training_data.shape}")
print(f"  Data type: {DATA_TYPE}")
print(f"  Range: [{tf.reduce_min(training_data):.3f}, {tf.reduce_max(training_data):.3f}]")

# Visualize the data
plt.figure(figsize=(8, 6))
plt.scatter(training_data[:, 0], training_data[:, 1], alpha=0.6, s=20)
plt.title(f'Training Data: {DATA_TYPE.replace("_", " ").title()}')
plt.xlabel('X₁')
plt.ylabel('X₂')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

print("Data preparation complete!")


**Model Configuration**

In [ ]:
# Model Configuration
print("Configuring quantum GAN architecture...")

# =============================================================================
# MODIFY THESE PARAMETERS FOR EXPERIMENTS
# =============================================================================

# Generator Configuration
GENERATOR_CONFIG = {
    'n_modes': 2,        # Number of quantum modes (output dimension)
    'latent_dim': 2,     # Latent noise dimension for input vector
    'layers': 2,         # Number of quantum layers
    'cutoff_dim': 8      # Fock space cutoff
}

# Discriminator Configuration
DISCRIMINATOR_CONFIG = {
    'n_modes': 1,        # Number of quantum modes
    'input_dim': 2,      # Input data dimension (should match generator output and training data)
    'layers': 1,         # Number of quantum layers
    'cutoff_dim': 8      # Fock space cutoff
}

# Training Configuration
TRAINING_CONFIG = {
    'epochs': 100,           # Number of training epochs
    'batch_size': 16,        # Batch size (keep small for quantum stability)
    'generator_lr': 1e-3,    # Generator learning rate
    'discriminator_lr': 1e-3, # Discriminator learning rate
    'beta1': 0.5,            # Adam optimizer beta1
    'beta2': 0.999,          # Adam optimizer beta2
    'monitor_interval': 10   # How often to compute quality metrics
}

# =============================================================================

print("Configuration:")
print(f"  Generator: {GENERATOR_CONFIG}")
print(f"  Discriminator: {DISCRIMINATOR_CONFIG}")
print(f"  Training: {TRAINING_CONFIG}")

# Create quantum components
print("\nCreating quantum components...")

generator = QuantumSFGenerator(**GENERATOR_CONFIG)
discriminator = QuantumSFDiscriminator(**DISCRIMINATOR_CONFIG)

print(f"✓ Generator created: {len(generator.trainable_variables)} trainable variables")
print(f"✓ Discriminator created: {len(discriminator.trainable_variables)} trainable variables")

# Create trainer
trainer = QGANSFTrainer(
    generator=generator,
    discriminator=discriminator,
    latent_dim=GENERATOR_CONFIG['latent_dim'],
    generator_lr=TRAINING_CONFIG['generator_lr'],
    discriminator_lr=TRAINING_CONFIG['discriminator_lr'],
    beta1=TRAINING_CONFIG['beta1'],
    beta2=TRAINING_CONFIG['beta2']
)

print(f"✓ Trainer created")
print(f"  Total parameters: {len(generator.trainable_variables) + len(discriminator.trainable_variables)}")

print("\nModel configuration complete!")


**Training Loop**

In [ ]:
# Training with monitoring
print("Starting quantum GAN training...")

# Training history storage
training_history = {
    'epochs': [],
    'generator_losses': [],
    'discriminator_losses': [],
    'quality_metrics': []
}

# Training loop
start_time = time.time()

for epoch in tqdm(range(TRAINING_CONFIG['epochs']), desc="Training"):
    # Train one epoch
    epoch_history = trainer.train(
        data=training_data,
        epochs=1,
        batch_size=TRAINING_CONFIG['batch_size'],
        verbose=False
    )
    
    # Store metrics
    if epoch % TRAINING_CONFIG['monitor_interval'] == 0:
        # Compute quality metrics
        z_test = tf.random.normal([200, GENERATOR_CONFIG['latent_dim']])
        generated_samples = generator.generate(z_test)
        
        real_mean = tf.reduce_mean(training_data, axis=0)
        gen_mean = tf.reduce_mean(generated_samples, axis=0)
        quality_metric = tf.norm(real_mean - gen_mean)
        
        # Store history
        training_history['epochs'].append(epoch)
        training_history['generator_losses'].append(float(epoch_history['g_loss'][-1]))
        training_history['discriminator_losses'].append(float(epoch_history['d_loss'][-1]))
        training_history['quality_metrics'].append(float(quality_metric))
        
        print(f"Epoch {epoch:3d}: G_loss={epoch_history['g_loss'][-1]:.4f}, "
              f"D_loss={epoch_history['d_loss'][-1]:.4f}, "
              f"Quality={quality_metric:.4f}")

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time:.1f}s")


**Results Visualization**

In [ ]:
# Training with monitoring
print("Starting quantum GAN training...")

# Training history storage
training_history = {
    'epochs': [],
    'generator_losses': [],
    'discriminator_losses': [],
    'quality_metrics': []
}

# Training loop
start_time = time.time()

for epoch in tqdm(range(TRAINING_CONFIG['epochs']), desc="Training"):
    # Train one epoch
    epoch_history = trainer.train(
        data=training_data,
        epochs=1,
        batch_size=TRAINING_CONFIG['batch_size'],
        verbose=False
    )
    
    # Store metrics
    if epoch % TRAINING_CONFIG['monitor_interval'] == 0:
        # Compute quality metrics
        z_test = tf.random.normal([200, GENERATOR_CONFIG['latent_dim']])
        generated_samples = generator.generate(z_test)
        
        real_mean = tf.reduce_mean(training_data, axis=0)
        gen_mean = tf.reduce_mean(generated_samples, axis=0)
        quality_metric = tf.norm(real_mean - gen_mean)
        
        # Store history
        training_history['epochs'].append(epoch)
        training_history['generator_losses'].append(float(epoch_history['g_loss'][-1]))
        training_history['discriminator_losses'].append(float(epoch_history['d_loss'][-1]))
        training_history['quality_metrics'].append(float(quality_metric))
        
        print(f"Epoch {epoch:3d}: G_loss={epoch_history['g_loss'][-1]:.4f}, "
              f"D_loss={epoch_history['d_loss'][-1]:.4f}, "
              f"Quality={quality_metric:.4f}")

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time:.1f}s")


In [ ]:
import matplotlib
matplotlib.use('Agg')  # or 'TkAgg' or 'Qt5Agg'
import matplotlib.pyplot as plt
%matplotlib inline

# Comprehensive results visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

epochs = training_history['epochs']

# 1. Loss Evolution
ax1 = axes[0, 0]
ax1.plot(epochs, training_history['generator_losses'], label='Generator Loss', linewidth=2)
ax1.plot(epochs, training_history['discriminator_losses'], label='Discriminator Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss Evolution')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# 2. Quality Metrics
ax2 = axes[0, 1]
ax2.plot(epochs, training_history['quality_metrics'], label='Mean Difference', color='green', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Quality Metric')
ax2.set_title('Generation Quality Evolution')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Final Generated vs Real Data
ax3 = axes[1, 0]
z_final = tf.random.normal([500, GENERATOR_CONFIG['latent_dim']])
final_generated = generator.generate(z_final)

ax3.scatter(training_data[:, 0], training_data[:, 1], alpha=0.6, s=20, label='Real Data')
ax3.scatter(final_generated[:, 0], final_generated[:, 1], alpha=0.6, s=20, label='Generated Data')
ax3.set_xlabel('X₁')
ax3.set_ylabel('X₂')
ax3.set_title('Final: Real vs Generated Data')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.axis('equal')

# 4. Training Summary
ax4 = axes[1, 1]
ax4.text(0.1, 0.8, f"Training Summary:", fontsize=14, fontweight='bold', transform=ax4.transAxes)
ax4.text(0.1, 0.7, f"Total Epochs: {TRAINING_CONFIG['epochs']}", fontsize=12, transform=ax4.transAxes)
ax4.text(0.1, 0.6, f"Final G Loss: {training_history['generator_losses'][-1]:.4f}", fontsize=12, transform=ax4.transAxes)
ax4.text(0.1, 0.5, f"Final D Loss: {training_history['discriminator_losses'][-1]:.4f}", fontsize=12, transform=ax4.transAxes)
ax4.text(0.1, 0.4, f"Final Quality: {training_history['quality_metrics'][-1]:.4f}", fontsize=12, transform=ax4.transAxes)
ax4.text(0.1, 0.3, f"Training Time: {total_time:.1f}s", fontsize=12, transform=ax4.transAxes)
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')

plt.tight_layout()
plt.suptitle('Quantum GAN Training Results', fontsize=16, y=1.02)
plt.show()

print("=" * 60)
print("QUANTUM GAN TRAINING COMPLETE")
print("=" * 60)
print("Results visualization displayed above.")
print("Modify parameters and re-run for different experiments!")
